# Langchain Supabase Setup

### 1. Loading the environment variables

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### 2. Initialize the Supabase Client

In [3]:
import os
from supabase.client import Client, create_client


supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)


### 3. Initialize embedding model

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import SupabaseVectorStore

embeddings = OpenAIEmbeddings()

### 4. Load the Data

In [31]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./data/demo.txt")
documents = loader.load()

### 5. Split the data into chunks

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, separators=["##"])
docs = text_splitter.split_documents(documents)

In [40]:
docs

[Document(page_content='## Life Summary:\nAshoka Maurya also known as Ashoka the Great was a very powerful Indian emperor from the Mauryan dynasty. He ruled almost all of the Indian subcontinent that is the entire region except some parts of present-day Tamil Nadu, Karnataka and Kerala during the period of 268 to 232 BCE. He is considered as one of the greatest rulers that India ever had. Though Ashoka was a very powerful emperor, one of the main reasons why he is respected and considered as great is because of the change that was brought or rather how he achieved enlightenment and changed for the better by realizing his mistakes. Ashoka was a very power-hungry ruler before he became a Buddhist. He plundered and conquered many powerful territories and while doing so caused immense amount of damage to both the place and the people living there. Several were left severely injured and many of them dead, but this all changed after he fought the battle for Kalinga. He waged a very bloody wa

### 6. Insert embedded data to supabase

In [42]:
vector_store = SupabaseVectorStore.from_documents(
    docs,
    embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
    chunk_size=500,
)

2023-11-03 14:08:14,418:INFO - HTTP Request: POST http://localhost:54321/rest/v1/documents "HTTP/1.1 201 Created"


> ### Querying by Similarity

In [49]:
query = "Who is Ashok Maurya?"
matched_docs = vector_store.similarity_search(query)
print(matched_docs[0].page_content)

2023-11-03 14:13:38,795:INFO - HTTP Request: POST http://localhost:54321/rest/v1/rpc/match_documents?limit=4 "HTTP/1.1 200 OK"


## Life Summary:
Ashoka Maurya also known as Ashoka the Great was a very powerful Indian emperor from the Mauryan dynasty. He ruled almost all of the Indian subcontinent that is the entire region except some parts of present-day Tamil Nadu, Karnataka and Kerala during the period of 268 to 232 BCE. He is considered as one of the greatest rulers that India ever had. Though Ashoka was a very powerful emperor, one of the main reasons why he is respected and considered as great is because of the change that was brought or rather how he achieved enlightenment and changed for the better by realizing his mistakes. Ashoka was a very power-hungry ruler before he became a Buddhist. He plundered and conquered many powerful territories and while doing so caused immense amount of damage to both the place and the people living there. Several were left severely injured and many of them dead, but this all changed after he fought the battle for Kalinga. He waged a very bloody war at Kalinga which just l

> ### Querying by Similarity with score

In [53]:
matched_docs = vector_store.similarity_search_with_relevance_scores(query)
print(matched_docs[0])

2023-11-03 14:16:30,935:INFO - HTTP Request: POST http://localhost:54321/rest/v1/rpc/match_documents?limit=4 "HTTP/1.1 200 OK"


(Document(page_content='## Life Summary:\nAshoka Maurya also known as Ashoka the Great was a very powerful Indian emperor from the Mauryan dynasty. He ruled almost all of the Indian subcontinent that is the entire region except some parts of present-day Tamil Nadu, Karnataka and Kerala during the period of 268 to 232 BCE. He is considered as one of the greatest rulers that India ever had. Though Ashoka was a very powerful emperor, one of the main reasons why he is respected and considered as great is because of the change that was brought or rather how he achieved enlightenment and changed for the better by realizing his mistakes. Ashoka was a very power-hungry ruler before he became a Buddhist. He plundered and conquered many powerful territories and while doing so caused immense amount of damage to both the place and the people living there. Several were left severely injured and many of them dead, but this all changed after he fought the battle for Kalinga. He waged a very bloody wa

### Instatiate Supabase Vector Store

In [ ]:
vector_store = SupabaseVectorStore(
    embedding=embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
)